In [1]:
import numpy as np

import tensorflow as tf

import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm


import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

from all_models import DavidNet

/home/fractaluser/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fractaluser/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fractaluser/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/fractaluser/a

resnext_model = ResNext_50()

resnext_model(tf.convert_to_tensor(np.random.normal(size=(32,32,3)).reshape(1,32,32,3).astype(np.float16)), 
              np.array([1]))

In [2]:
import run_util
from run_util import Run

from importlib import reload
reload(run_util)

<module 'run_util' from '/home/fractaluser/Personal/Narahari/eva_research_team4/src/run_util.py'>

In [3]:
BATCH_SIZE = 32 #@param {type:"integer"}
MOMENTUM = 0.95 #@param {type:"number"}

MIN_MOMENTUM = 0.8 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 2 #@param {type:"integer"}


MIN_LEARNING_RATE = 0.000001 #@param {type:"number"}

END_LR_SMOOTHING_PERC = 0.15 #@param {type:"number"}

COMMENTS = "Integration tests" #@param {type:"string"}


params_tune = {
    
  "epochs":EPOCHS, 
 
  "batch_size" : BATCH_SIZE,

  "max_lr": LEARNING_RATE,

  "min_lr":MIN_LEARNING_RATE,

  "end_anneal_pc":END_LR_SMOOTHING_PERC,

  "max_mom":MOMENTUM,
 
  "min_mom":MIN_MOMENTUM,
 
  "wd":WEIGHT_DECAY,
  
  "skip_testing_epochs":0,
    
  "batches_per_epoch":100//BATCH_SIZE,
    
  "comments":COMMENTS
}

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train_org = x_train

x_test_org = x_test

len_train, len_test = len(x_train), len(x_test)

y_train=y_train_org = y_train.astype('int64').reshape(len_train)

y_test_org = y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))

train_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float16') # todo: check here

pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

#x_train = normalize(pad4(x_train))
x_test_org = x_test = normalize(x_test).astype(np.float16)

In [5]:
def tst_data_supplier(epoch_num):  
  
  global x_test_org
  
  global y_test
  
  return (x_test_org[:100,:,:,:], y_test_org[:100])

def trn_data_supplier(epoch_num):
  
  global y_train_org
    
  global x_train_org

  return (normalize(x_train_org[:100,:,:,:]).astype(np.float16), y_train_org[:100])
  
  #return ((globals()['x_train%s' % epoch_num]).astype(np.float16), y_train_org)  

In [6]:
model_fn = DavidNet

obj = Run()

obj.run(model_fn, params_tune, trn_data_supplier, tst_data_supplier)

****training starts****


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
epoch: 1 lr: 0.04000000000000001 train loss: 2.348 train acc:0.130 val loss: 2.312 val acc: 0.100 time:57.789
epoch: 2 lr: 0.020000500000000004 train loss: 2.344 train acc:0.130 val loss: 2.316 val acc: 0.100 time:115.282

   total_model_parameters  epochs  batch_size  max_lr    min_lr  anneal_perc  \
0                  343264       2          32     0.4  0.000001         0.15   

   max_mom  min_mom  time_taken  train_acc  test_acc      wd  train_loss  \
0     0.95      0.8  115.288906       0.13       0.1  0.0005    2.344219   

   test_loss                                               runs  \
0   2.316484  {'epoch': [1, 2], 'time': [57.78938937187195, ...   

              log_time           comments  
0  2019-08-31 04:11:53  Integration tests  


 115.30396485328674,
 0.13,
 0.1,
 2.34421875,
 2.316484375]

In [7]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [ ]:
obj.lr_finder(model_fn, lr_list=[0.001, 0.01, 0.1, 1])

running lr_finder


1
****training starts****



2
****training starts****



3
****training starts****



4
****training starts****
